<font color=#163e5a>  
    <h1>Preprocessing the data</h1>
</font>

!python D:/Documents/PythonScripts/PhDScripts/Pt_p2/preprocess_bcdi.py

## Preprocess output

**Two .npz files**
* the mask
* the (normalised?) detector data

Pt_p2/pynxraw/S1398_ampdispstrain_gap_iso0.2_mode_avg3_apodize_blackman_lab-frame.npz, probably final output ?

<font color=#163e5a>  
    <h1>Find the .npz files</h1>
    <a href = "https://www.pytables.org/cookbook/inmemory_hdf5_files.html" target = "_blank">Documentation</a>
</font>

In [12]:
%%bash
ls Pt_p2/pynxraw/*.npz

Pt_p2/pynxraw/Mask_10_100_0_1.npz
Pt_p2/pynxraw/S1398_maskpynx_norm_128_300_294_1_1_1.npz
Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1.npz


In [3]:
%run -i Scripts/PlotNPZ

Path to file from current folder: Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1.npz
Stored arrays :['data']


interactive(children=(Dropdown(description='Array used:', options=('data',), style=DescriptionStyle(descriptio…

<font color=#163e5a>  
    <h1>Find the .cxi files</h1>
    <a href = "https://raw.githubusercontent.com/cxidb/CXI/master/cxi_file_format.pdf" target = "_blank">Documentation</a>
</font>

In [14]:
%%bash
ls Pt_p2/pynxraw/*.cxi

Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-25-40_Run0008_LLKf000.1159_LLK000.1328_SupportThreshold0.29541.cxi
Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-26-53_Run0010_LLKf000.1158_LLK000.1330_SupportThreshold0.29706.cxi
Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-37-13_Run0027_LLKf000.1157_LLK000.1329_SupportThreshold0.29221.cxi
Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-39-39_Run0031_LLKf000.1158_LLK000.1330_SupportThreshold0.28916.cxi
Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-41-28_Run0034_LLKf000.1158_LLK000.1332_SupportThreshold0.28754.cxi
Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-44-30_Run0039_LLKf000.1158_LLK000.1330_SupportThreshold0.29166.cxi
Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-47-34_Run0044_LLKf000.1158_LLK000.1329_SupportThreshold0.29538.cxi
Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-59-06_Run0063_LLKf000.1155_LLK000.1

* The last .cxi file contains only the detector and the mask data

* All the other .cxi files are in real space and show the **complex** electronic density, they each are the output of the phase retrieval process. The image size in real space is also given.
* The support is linked to the mask, seems that it must be a binary array (0 masks, 1 does not mask)
* Free pixel mask is not the original .npz file ? Boolean array maybe fixes which pixels cannot be assigned to the recalculated intensity, final one ?
* entry_last is just a link to entry_1, idk why, doubles data size ? not ure if just link

In [3]:
%run -i Scripts/PlotCXI

Path to file from current folder: Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-25-40_Run0008_LLKf000.1159_LLK000.1328_SupportThreshold0.29541.cxi
Data file architecture :

Pt_p2/pynxraw/S1398_pynx_norm_128_300_294_1_1_1-2020-12-04T19-25-40_Run0008_LLKf000.1159_LLK000.1328_SupportThreshold0.29541.cxi (File) ''
Last modif.: 'Fri Dec  4 19:26:17 2020'
Object Tree: 
/ (RootGroup) ''
/cxi_version (Array()) ''
/entry_last (SoftLink) -> /entry_1
/entry_1 (Group) ''
/entry_1/program_name (UnImplemented()) ''
/entry_1/start_time (UnImplemented()) ''
/entry_1/data_1 (Group) ''
/entry_1/data_1/data (SoftLink) -> /entry_1/image_1/data
/entry_1/image_1 (Group) ''
/entry_1/image_1/data (CArray(128, 300, 294), shuffle, zlib(4)) ''
/entry_1/image_1/data_space (UnImplemented()) ''
/entry_1/image_1/data_type (UnImplemented()) ''
/entry_1/image_1/image_size (Array(3,)) ''
/entry_1/image_1/mask (CArray(128, 300, 294), shuffle, zlib(4)) ''
/entry_1/image_1/support (SoftLink) -> /entry_1/im

interactive(children=(Dropdown(description='First 2 axes:', index=1, options=('xy', 'yz', 'xz'), style=Descrip…

# Try to extract a support determination from the 10 runs

In [11]:
def CreateSupport(Path, Threshold):

    with tb.open_file(Path, "r") as f:
        print(f"\n{Path}")
        # Since .cxi files follow a specific architectture, we know where our data is.
        mask = f.root.entry_1.image_1.mask[:]
        print(f"All elements in mask array are equal to 1 : {np.all(mask == 1)}")
        print(f"Shape of support array {np.shape(mask)}")

        # support = f.root.entry_1.image_1.support[:]
        # print(f"All elements in support array are equal to 1 : {np.all(support == 1)}")

        ED = f.root.entry_1.data_1.data[:]
        print(f"Shape of real space complex electronic density array {np.shape(ED)}")

        # Find max value of real space image, we work with the amplitude
        Amp = np.abs(ED)
        MaxRS = Amp.max()
        print(f"Maximum value in amplitude array: {MaxRS}")
        
        result = np.where(Amp < Threshold * MaxRS, 0, 1)
        # Check if the indices we have seem to be around the center of the array
        rocc = np.where(result == 1)
        rnocc = np.where(result == 0)
        
        print("Indices where result == 1:\n",rocc)
        print(f"Percentage of 3D array occupied by support:\n{np.shape(rocc)[1] / np.shape(rnocc)[1]}")

        return result

In [12]:
import glob
Path = sorted(glob.glob("Pt_p2\\pynxraw\\S1398_pynx_norm_128_300_294_1_1_1-2020*cxi"))
t = 0.20

TenSupports = [CreateSupport(p,t) for p in Path]

# 3D array of 1 and 0 -> mask

np.shape(TenSupports[0])

a = TenSupports[0]

for t in TenSupports[1:]:
    a = np.add(a, t)
    
mask = np.where(a > 4, 1, 0)
np.savez("Mask_10_100_0_1.npz", mask = mask)


Pt_p2\pynxraw\S1398_pynx_norm_128_300_294_1_1_1-2020-12-03T10-44-11_Run0019_LLKf000.1524_LLK4828897416.5916_SupportThreshold0.14254.cxi
All elements in mask array are equal to 1 : True
Shape of support array (128, 300, 294)
Shape of real space complex electronic density array (128, 300, 294)
Maximum value in amplitude array: 21.619176864624023
Indices where result == 1:
 (array([35, 35, 35, ..., 97, 97, 97], dtype=int64), array([136, 137, 138, ..., 163, 163, 163], dtype=int64), array([151, 151, 155, ..., 141, 142, 143], dtype=int64))
Percentage of 3D array occupied by support:
0.007032888772175579

Pt_p2\pynxraw\S1398_pynx_norm_128_300_294_1_1_1-2020-12-03T10-44-45_Run0020_LLKf000.1371_LLK5106613636.0168_SupportThreshold0.10094.cxi
All elements in mask array are equal to 1 : False
Shape of support array (128, 300, 294)
Shape of real space complex electronic density array (128, 300, 294)
Maximum value in amplitude array: 21.28377342224121
Indices where result == 1:
 (array([ 36,  36,  

Need to smooth by convoluting with a gaussian

In [13]:
rocc = np.where(mask == 1)
rnocc = np.where(mask == 0)

print("Indices where result == 1:\n",rocc)
print(f"Percentage of 3D array occupied by support:\n{np.shape(rocc)[1] / np.shape(rnocc)[1]}")

Indices where result == 1:
 (array([35, 35, 35, ..., 95, 95, 95], dtype=int64), array([138, 139, 140, ..., 150, 150, 153], dtype=int64), array([155, 155, 153, ..., 139, 140, 140], dtype=int64))
Percentage of 3D array occupied by support:
0.007585090853659189


<font color=#163e5a>  
    <h1>Create modes</h1>
    <p>Only on lid01 because pynx package</p>
</font>

! python pynx-cdi-analysis.py S1398_pynx_norm_128_300_294_1_1_1-* modes=1

<font color=#163e5a>  
    <h1>Launch strain analysis</h1>
    <p>Only on lid01 because local bcdi package has problems</p>
</font>

In [16]:
!python D:/Documents/PythonScripts/PhDScripts/Pt_p2/pynxraw/strain.py

^C
